# 1. Make Some Detections

In [84]:
import pafy
import cv2
import mediapipe as mp
import numpy as np
import os

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [85]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for i in os.listdir('./push_up/'):
        path = './push_up/' + i
        frame = cv2.imread(path, cv2.IMREAD_COLOR)
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV 

In [90]:
import csv
import os
import numpy as np
import glob

In [91]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [92]:
landmarks = ['class'] #landmark의 각 이름을 다르게 설정
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [93]:
#csv로 만들기- 미리 파일과 컬럼을 지정해야함
with open('dataset.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [118]:
class_name = "Push_up"

In [119]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
     for i in os.listdir('./push_up/'):
        path = './push_up/' + i
        frame = cv2.imread(path, cv2.IMREAD_COLOR)
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = holistic.process(image)
   

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
         
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            pose_row.insert(0,class_name)
            
            with open('dataset.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row)
        
        except:
            pass
            
                
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3-1. Read in collected Data and Process

In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [120]:
df = pd.read_csv('dataset.csv')

In [121]:
df[df['class']=='Push_up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
2226,Push_up,0.168150,0.460982,-0.050383,0.998749,0.152255,0.442348,-0.086713,0.998843,0.153245,...,0.293615,0.749672,0.897344,0.900246,-0.230953,0.972185,0.870396,0.851225,0.197830,0.800784
2227,Push_up,0.057507,0.675714,0.104270,0.998806,0.048234,0.653924,0.070495,0.998882,0.048867,...,0.147567,0.727947,0.890659,0.912623,-0.337812,0.967096,0.866984,0.877444,0.062661,0.769174
2228,Push_up,0.060873,0.752542,0.150541,0.998805,0.049866,0.740561,0.102692,0.998861,0.050058,...,0.185102,0.709875,0.889971,0.913673,-0.387465,0.963829,0.867111,0.873304,0.106524,0.749163
2229,Push_up,0.089080,0.532914,0.040302,0.998876,0.074047,0.519582,-0.015604,0.998931,0.074532,...,0.360500,0.720472,0.883004,0.914061,-0.343357,0.965381,0.863340,0.873231,0.269978,0.759185
2230,Push_up,0.148152,0.506032,-0.026416,0.998935,0.127841,0.492706,-0.072046,0.998982,0.126795,...,0.460067,0.725083,0.881014,0.913392,-0.291813,0.965599,0.852629,0.841137,0.409133,0.761061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,Push_up,0.117789,0.477687,-0.033280,0.998681,0.101811,0.463559,-0.077825,0.998761,0.101847,...,0.378650,0.709955,0.893010,0.912092,-0.216123,0.965215,0.863331,0.851048,0.302212,0.745605
2848,Push_up,0.078852,0.741090,0.106218,0.998667,0.063783,0.732937,0.067811,0.998746,0.064043,...,0.143135,0.704863,0.892399,0.914082,-0.262348,0.964569,0.863386,0.857506,0.087710,0.739330
2849,Push_up,0.084374,0.759358,0.124647,0.998698,0.068755,0.748955,0.083471,0.998776,0.068837,...,0.116208,0.700191,0.892133,0.913917,-0.321193,0.965585,0.863672,0.857677,0.058050,0.739242
2850,Push_up,0.104024,0.528328,-0.018828,0.998760,0.085878,0.511313,-0.063387,0.998835,0.085669,...,0.325262,0.706779,0.891891,0.913805,-0.331549,0.965325,0.862178,0.856934,0.238383,0.742997


In [122]:
X = df.drop('class',axis=1) #features- 클래스열을 제외한 모든 단일 값
y = df['class'] #target value

In [123]:
# Partition
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=1234)

## 3-2. Train Machine Learning Classification Model

In [124]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [125]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}



In [126]:
# model training
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [133]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [127]:
fit_models['rf'].predict(X_test)

array(['L_fly', 'L_fly', 'Lunge', 'Squat', 'L_fly', 'One_leg', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'L_fly', 'Squat', 'L_fly', 'L_fly',
       'Lunge', 'Squat', 'Squat', 'Push_up', 'One_leg', 'Squat',
       'Push_up', 'Push_up', 'Squat', 'Lunge', 'One_leg', 'Push_up',
       'Squat', 'Dumb_bell', 'Push_up', 'Squat', 'Lunge', 'Squat',
       'Push_up', 'Squat', 'One_leg', 'One_leg', 'Push_up', 'L_fly',
       'L_fly', 'L_fly', 'Dumb_bell', 'Dumb_bell', 'Lunge', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'Push_up', 'L_fly', 'Lunge',
       'Lunge', 'Dumb_bell', 'Lunge', 'Dumb_bell', 'Squat', 'One_leg',
       'L_fly', 'Lunge', 'One_leg', 'Dumb_bell', 'L_fly', 'One_leg',
       'Push_up', 'Squat', 'Dumb_bell', 'Dumb_bell', 'Squat', 'Dumb_bell',
       'One_leg', 'One_leg', 'Push_up', 'Push_up', 'Lunge', 'Dumb_bell',
       'Push_up', 'L_fly', 'Squat', 'L_fly', 'Squat', 'Squat', 'Lunge',
       'Lunge', 'Squat', 'One_leg', 'Squat', 'Dumb_bell', 'Dumb_bell',
       'Push_up',

In [130]:
fit_models['lr'].predict(X_test)

array(['L_fly', 'L_fly', 'Lunge', 'Squat', 'L_fly', 'One_leg', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'L_fly', 'Squat', 'L_fly', 'L_fly',
       'Lunge', 'Squat', 'Squat', 'Push_up', 'One_leg', 'Squat',
       'Push_up', 'Push_up', 'Squat', 'Lunge', 'One_leg', 'Push_up',
       'Squat', 'Dumb_bell', 'Push_up', 'Squat', 'Lunge', 'Squat',
       'Push_up', 'Squat', 'One_leg', 'One_leg', 'Push_up', 'L_fly',
       'L_fly', 'L_fly', 'Dumb_bell', 'Dumb_bell', 'Lunge', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'Push_up', 'L_fly', 'Lunge',
       'Lunge', 'Dumb_bell', 'Lunge', 'Dumb_bell', 'Squat', 'One_leg',
       'L_fly', 'Lunge', 'One_leg', 'Dumb_bell', 'L_fly', 'One_leg',
       'Push_up', 'Squat', 'Dumb_bell', 'Dumb_bell', 'Squat', 'Dumb_bell',
       'One_leg', 'One_leg', 'Push_up', 'Push_up', 'Lunge', 'Dumb_bell',
       'Push_up', 'L_fly', 'Squat', 'L_fly', 'Squat', 'Squat', 'Lunge',
       'Lunge', 'Squat', 'One_leg', 'Squat', 'Dumb_bell', 'Dumb_bell',
       'Push_up',

In [131]:
fit_models['rc'].predict(X_test)

array(['L_fly', 'L_fly', 'Lunge', 'Squat', 'L_fly', 'One_leg', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'L_fly', 'Squat', 'L_fly', 'L_fly',
       'Lunge', 'Squat', 'Squat', 'Push_up', 'One_leg', 'Squat',
       'Push_up', 'Push_up', 'Squat', 'Lunge', 'One_leg', 'Push_up',
       'Squat', 'Dumb_bell', 'Push_up', 'Squat', 'Lunge', 'Squat',
       'Push_up', 'Squat', 'One_leg', 'One_leg', 'Push_up', 'L_fly',
       'L_fly', 'L_fly', 'Dumb_bell', 'Dumb_bell', 'Lunge', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'Push_up', 'L_fly', 'Lunge',
       'Lunge', 'Dumb_bell', 'Lunge', 'Dumb_bell', 'Squat', 'One_leg',
       'L_fly', 'Lunge', 'One_leg', 'Dumb_bell', 'L_fly', 'One_leg',
       'Push_up', 'Squat', 'Dumb_bell', 'Dumb_bell', 'Squat', 'Dumb_bell',
       'One_leg', 'One_leg', 'Push_up', 'Push_up', 'Lunge', 'Dumb_bell',
       'Push_up', 'L_fly', 'Squat', 'L_fly', 'Squat', 'Squat', 'Lunge',
       'Lunge', 'Squat', 'One_leg', 'Squat', 'Dumb_bell', 'Dumb_bell',
       'Push_up',

## 3-3. Evaluate and Serialize Model

In [128]:
from sklearn.metrics import accuracy_score #accuracy
import pickle #model을 디스크에 저장

In [134]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) #predicted result store
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.9988317757009346


In [136]:
fit_models['rf'].predict(X_test)

array(['L_fly', 'L_fly', 'Lunge', 'Squat', 'L_fly', 'One_leg', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'L_fly', 'Squat', 'L_fly', 'L_fly',
       'Lunge', 'Squat', 'Squat', 'Push_up', 'One_leg', 'Squat',
       'Push_up', 'Push_up', 'Squat', 'Lunge', 'One_leg', 'Push_up',
       'Squat', 'Dumb_bell', 'Push_up', 'Squat', 'Lunge', 'Squat',
       'Push_up', 'Squat', 'One_leg', 'One_leg', 'Push_up', 'L_fly',
       'L_fly', 'L_fly', 'Dumb_bell', 'Dumb_bell', 'Lunge', 'Lunge',
       'Push_up', 'Push_up', 'Lunge', 'Push_up', 'L_fly', 'Lunge',
       'Lunge', 'Dumb_bell', 'Lunge', 'Dumb_bell', 'Squat', 'One_leg',
       'L_fly', 'Lunge', 'One_leg', 'Dumb_bell', 'L_fly', 'One_leg',
       'Push_up', 'Squat', 'Dumb_bell', 'Dumb_bell', 'Squat', 'Dumb_bell',
       'One_leg', 'One_leg', 'Push_up', 'Push_up', 'Lunge', 'Dumb_bell',
       'Push_up', 'L_fly', 'Squat', 'L_fly', 'Squat', 'Squat', 'Lunge',
       'Lunge', 'Squat', 'One_leg', 'Squat', 'Dumb_bell', 'Dumb_bell',
       'Push_up',

In [137]:
y_test

1766        L_fly
2135        L_fly
1500        Lunge
368         Squat
1737        L_fly
          ...    
1912        L_fly
1010    Dumb_bell
497       One_leg
1242        Lunge
2349      Push_up
Name: class, Length: 856, dtype: object

In [138]:
# best_model저장
with open('fitness_pose.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## 3-4. Make Detections with Model

In [139]:
with open('fitness_pose.pkl', 'rb') as f: #모델 로드
    model = pickle.load(f)

In [140]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [144]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    
    while cap.isOpened(): 
        ret,frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = holistic.process(image)
   

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
         
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Model Detection
            X = pd.DataFrame([pose_row]) #class를 frame에 보여줌
            fitness_pose_class = model.predict(X)[0] #클래스 예측
            fitness_pose_prob = model.predict_proba(X)[0] # 확률예측
            print(fitness_pose_class, fitness_pose_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmark[mp_pose.poseLandmark.LEFT_EAR].x,
                                    results.pose_landmarks.landmark[mp_pose.poseLandmark.LEFT_EAR].y,)
                            ), [640,480]).astype(int))
            
            cv2.rectangle(image,
                        (coords[0], coords[1]+5),
                        (coords[0]+len(fitness_pose_class)*20, coords[1]-30),
                        (245,117,16),-1)
            cv2.putText(image, fitness_pose_class, coords, 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0),(250,60),(245,117,16),-1)
            
            # Display Class
            cv2.putText(image, 'Class'
                       , (95,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1, cv2.LINE_AA)
            cv2.putText(image, fitness_pose_class.split(' ')[0]
                       , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'Prob'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1, cv2.LINE_AA)
            cv2.putText(image, str(round(fitness_pose_prob[np.argmax(fitness_pose_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2, cv2.LINE_AA)
        
        except:
            pass
            
                
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

L_fly [0.09 0.47 0.12 0.   0.24 0.08]
L_fly [0.1  0.4  0.12 0.01 0.29 0.08]
L_fly [0.1  0.39 0.12 0.01 0.29 0.09]
L_fly [0.09 0.39 0.11 0.01 0.29 0.11]
L_fly [0.1  0.39 0.11 0.01 0.29 0.1 ]
L_fly [0.1  0.39 0.11 0.01 0.29 0.1 ]
L_fly [0.1  0.4  0.11 0.01 0.29 0.09]
L_fly [0.09 0.39 0.08 0.01 0.34 0.09]
L_fly [0.09 0.39 0.08 0.01 0.34 0.09]
L_fly [0.08 0.37 0.08 0.02 0.37 0.08]
L_fly [0.09 0.39 0.07 0.02 0.36 0.07]
L_fly [0.08 0.39 0.05 0.01 0.37 0.1 ]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.01 0.32 0.12]
L_fly [0.09 0.4  0.06 0.02 0.33 0.1 ]
L_fly [0.08 0.39 0.05 0.02 0.37 0.09]
L_fly [0.08 0.39 0.05 0.01 0.37 0.1 ]
L_fly [0.08 0.39 0.05 0.02 0.37 0.09]
L_fly [0.08 0.39 0.05 0.02 0.37 0.09]
L_fly [0.08 0.39 0.05 0.02 0.37 0.09]
L_fly [0.08 0.39 0.05 0.02 0.37 0.09]
L_fly [0.08 